# Summary
1. We select the built class of Google Dynamic World (GDW) and filter the certainty to be greater than 0.06, as the highest false positive in the grassland had a certainty of 0.0569.
2. We also mask resulting GDW layer with a rock mask, which we manually drew from false positives from the Google Dynamic World built class.
3. We import Google Open Buildings, 2022 (GOB), convert it to raster and combine the GDW.
4. Splitting up Africa in blocks, we then export the resulting combined built layer to Google Cloud Storage.

### Future
- Conduct similar analysis for all years
- Improve years before GDW (2022)

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

In [ ]:
block_list = list(range(1, 26))

blocks = []
with open("/content/drive/MyDrive/data/blocks.geojson") as f:
    json_data = json.load(f)
    for block_id in block_list:
        for feature in json_data['features']:
            if feature['properties']['id'] == block_id:
                feature['properties']['style'] = {}
                blocks.append(feature)

ee_blocks = [geemap.geojson_to_ee(block) for block in blocks]

In [ ]:
# Import rockMask from GoogleCloud
fs = gcsfs.GCSFileSystem(project='nature-watch-387210')

with fs.open('gs://nature-watch-bucket/vector/built_rock_mask.geojson', 'r') as f:
    data = json.load(f)

rocks = geemap.geojson_to_ee(data)

In [ ]:
# Specify dates
year = 2022
start_date = '{}-01-01'.format(year)
end_date = '{}-01-01'.format(year + 1)


In [ ]:
CLOUD_FILTER = 70
CLD_PRB_THRESH = 5
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


def get_s2_sr_cld_col(datefilter):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filter(datefilter)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filter(datefilter)
        )

    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


date_filter = ee.Filter.date(ee.Date(start_date), ee.Date(end_date))
sentinel = get_s2_sr_cld_col(date_filter).map(add_cld_shdw_mask)


print(sentinel.first().bandNames().getInfo())

In [ ]:
# Mask Dynamic World

def mask_dynamicworld_by_sentinel(dw_image):
    # Get the date of the DYNAMICWORLD image
    dw_date = dw_image.date()

    # Filter the Sentinel collection to get the corresponding image
    sentinel_mask_img = sentinel.filterDate(dw_date.advance(-0.5, 'day'), dw_date.advance(0.5, 'day')).median()

    # Ensure there's a matching image in the Sentinel collection
    if not sentinel_mask_img:
        return dw_image

    # Get the cloudmask from the Sentinel image and apply it to the DYNAMICWORLD image
    cloudmask = sentinel_mask_img.select('cloudmask').Not()
    return dw_image.updateMask(cloudmask)

# Apply the mask to each image in the DYNAMICWORLD collection
people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date)
masked_people = people.map(mask_dynamicworld_by_sentinel)



In [ ]:
# Prepare layers

# Google Dynamic World
people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('label').eq(6).selfMask()
certainty_mask = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').gt(0.06).selfMask()
best_people = people.mask(certainty_mask).eq(1).updateMask(cloud_mask).selfMask().unmask()

# Use the rock mask
raster_mask = ee.Image.constant(1).paint(rocks, 0)
people_rock_masked = best_people.mask(raster_mask).unmask(0).selfMask()

# Google Open Buildings
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v2/polygons').filter('confidence >= 0.70');
buildings_raster = buildings.reduceToImage(
  properties=['confidence'],
  reducer=ee.Reducer.median()
).gt(0).selfMask().select(['median'], ['label'])

# Join with other layers
built = people_rock_masked.unmask(0).add(buildings_raster.unmask(0)).gt(0).selfMask()


In [ ]:
# Loop through blocks and export
for id, block in enumerate(ee_blocks):
  image_name = 'built2022_' + str(id)
  fileNamePrefix = 'COGS/built/built2022/' + image_name


  exportConfig = {
      'image': built,
      'description': image_name,
      'bucket': 'nature-watch-bucket',
      'fileNamePrefix': fileNamePrefix,
      'scale': 30,
      'maxPixels': 3147395000,
      'region': block,
      'fileFormat': 'GeoTIFF',
      'formatOptions': {'cloudOptimized': True}
  }

  task = ee.batch.Export.image.toCloudStorage(**exportConfig)
  task.start()

In [ ]:
ee.data.listOperations()

In [ ]:
certainty = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built')


visDW = {'min': 0, 'max': 8, 'palette': [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1']}

Map = geemap.Map()
Map.add_basemap('SATELLITE')
# Map.addLayer(certainty, {}, 'certainty')
Map.addLayer(masked_people.median().select('label'), visDW, 'masked_people')

Map.setCenter(11.475, 1.604, 12)
Map